# Extraindo empresas

## Extraindo todas empresas do Brasil da categoria BESST

In [ ]:
import requests
import yfinance as yf
import pandas as pd
import os

# --- PASSO 1: CONFIGURAÇÃO DOS SETORES-ALVO ---
setores_alvo = [
    "Finance",
    "Utilities",
    "Communications",
    "Industrial Services"
]

# --- FUNÇÃO PRINCIPAL COM A LÓGICA FINAL ---
def scanner_final_com_tipo(setores):
    print("Iniciando scanner final focado (filtrando por tipo 'stock' e 'fund')...")
    try:
        url = "https://brapi.dev/api/quote/list"
        lista_completa_brapi = requests.get(url).json().get('stocks', [])
    except Exception as e:
        print(f"❌ Erro fatal ao buscar a lista da brapi: {e}")
        return {}

    print("Mapeando ativos dos setores alvo...")
    tickers_para_analisar = {}
    for stock in lista_completa_brapi:
        setor_brapi = stock.get('sector')
        tipo_ativo = stock.get('type')
        if setor_brapi in setores and tipo_ativo in ['stock', 'fund']:
            ticker = f"{stock.get('stock')}.SA"
            tickers_para_analisar[ticker] = {
                'setor': setor_brapi,
                'logo': stock.get('logo'),
                'tipo': tipo_ativo 
            }
    
    total = len(tickers_para_analisar)
    print(f"✅ Mapeamento concluído. {total} ativos dos tipos 'stock' e 'fund' serão analisados.")

    dados_finais = {}
    print("\nIniciando coleta detalhada de dados no yfinance...")
    for i, (ticker, brapi_data) in enumerate(tickers_para_analisar.items()):
        try:
            print(f"Analisando [{i+1}/{total}]: {ticker}...")
            stock = yf.Ticker(ticker)
            info = stock.info
            
            if info.get("country") != "Brazil":
                print(f"  -> DESCARTADO: {ticker} não é do Brasil.")
                continue
            
            dados_finais[ticker] = {
                "Empresa": info.get("longName", "N/A"),
                "Setor (brapi)": brapi_data['setor'],
                "Subsetor (yfinance)": info.get("industry", "N/A"),
                "País": info.get("country", "N/A"),
                "Tipo": brapi_data['tipo'],
                "Market Cap": info.get("marketCap"),
                "Logo": brapi_data['logo'],
            }
            print(f"  -> ✅ INCLUÍDO: {ticker}")

        except Exception as e:
            print(f"  -> ERRO: Falha ao processar {ticker}. Erro: {e}")
            continue
                
    return dados_finais

# --- EXECUÇÃO E APRESENTAÇÃO DOS RESULTADOS ---
resultados = scanner_final_com_tipo(setores_alvo)

if resultados:
    print("\n✅ Análise finalizada.")
    df_resultado = pd.DataFrame.from_dict(resultados, orient="index")
    
    df_resultado.reset_index(inplace=True)
    df_resultado.rename(columns={'index': 'Ticker'}, inplace=True)
    
    if "Market Cap" in df_resultado.columns:
         df_resultado["Market Cap Num"] = pd.to_numeric(df_resultado["Market Cap"], errors='coerce')
         df_resultado["Market Cap"] = df_resultado["Market Cap Num"].apply(
             lambda x: f"R$ {x/1_000_000_000:.2f} Bi" if pd.notna(x) and x > 0 else "N/A"
         )

    df_ordenado = df_resultado.sort_values(by=['Setor (brapi)', 'Market Cap Num'], ascending=[True, False])
    
    colunas_finais = ['Ticker', 'Empresa', 'Setor (brapi)', 'Subsetor (yfinance)', 'País', 'Tipo', 'Market Cap', 'Logo']
    df_final = df_ordenado[colunas_finais]
    
    print("\n--- Relatório Final (Ações e Fundos) ---")
    print(df_final.to_string(index=False))

    # --- SALVANDO O ARQUIVO NO CAMINHO ESPECÍFICO ---
    try:
        output_folder = r"E:\finance-manager\data"
        output_filename = "scanner_acoes_e_fundos_filtrado.csv"
        full_path = os.path.join(output_folder, output_filename)
        
        os.makedirs(output_folder, exist_ok=True)
        
        df_final.to_csv(full_path, index=False)
        print(f"\n✅ Resultados salvos com sucesso no caminho: {full_path}")
    except Exception as e:
        print(f"\n❌ Erro ao salvar o arquivo CSV: {e}")
else:
    print("\nNenhuma empresa foi encontrada para os critérios especificados.")

In [ ]:
# import pandas as pd

# df_dados2 = pd.read_excel(r'E:\Github\finance-manager\datasets\scanner_acoes_filtrado.xlsx')

# print(df_dados2)

# Transformação

In [4]:
import yfinance as yf
import pandas as pd
import os
from datetime import datetime, date
from dateutil.relativedelta import relativedelta

# --- PASSO 1: LER A LISTA DE ATIVOS DO SEU ARQUIVO ---
arquivo_input = r"E:\finance-manager\data\scanner_acoes_e_fundos_filtrado.csv"

try:
    print(f"Lendo a lista de ativos do arquivo '{arquivo_input}'...")
    df_input = pd.read_csv(arquivo_input)
    colunas_esperadas = ['Ticker', 'Empresa', 'Setor (brapi)', 'País', 'Tipo', 'Market Cap', 'Logo']
    colunas_faltantes = [col for col in colunas_esperadas if col not in df_input.columns]
    if colunas_faltantes:
        print(f"⚠️ Colunas Faltantes: {colunas_faltantes}")

    tickers_para_analisar = df_input['Ticker'].tolist()
    print(f"✅ {len(tickers_para_analisar)} ativos serão analisados.")

except FileNotFoundError:
    print(f"❌ Arquivo '{arquivo_input}' não encontrado.")
    exit()
except Exception as e:
    print(f"❌ Erro ao ler o arquivo: {e}")
    exit()

# --- PASSO 2: BUSCAR OS DADOS DETALHADOS COM LÓGICA CONDICIONAL ---
dados_detalhados_acoes = {}
erros = []
print("\nColetando dados detalhados...")

for ticker in tickers_para_analisar:
    print(f"Processando: {ticker}...", end="")
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        history = stock.history(period="5y")

        info_original = df_input[df_input['Ticker'] == ticker].iloc[0]
        setor_principal = info_original.get('Setor (brapi)', 'N/A')

        # --- DADOS COMUNS A TODOS OS SETORES ---
        dy_pela_taxa = "N/A"
        try:
            taxa_anual_div = info.get("trailingAnnualDividendRate")
            preco_atual = info.get("currentPrice")
            if taxa_anual_div is not None and preco_atual is not None and preco_atual > 0:
                dy_pela_taxa = (taxa_anual_div / preco_atual) * 100
        except:
            dy_pela_taxa = "N/A"

        # Crescimento Preço (%) baseado em histórico de preço
        growth_price = (history['Close'].iloc[-1] / history['Close'].iloc[0] - 1) * 100 if len(history) > 1 else 0

        # Mapeamento de Tipo
        tipo_map = {"stock": "Ações", "fund": "Fundos"}
        tipo = tipo_map.get(info_original.get('Tipo', 'N/A'), info_original.get('Tipo', 'N/A'))

        dados_base = {
            "Empresa": info_original.get('Empresa', 'N/A'),
            "Setor (brapi)": setor_principal,
            "Tipo": tipo,
            "Market Cap (R$)": info_original.get('Market Cap', 'N/A'),
            "Logo": info_original.get('Logo', 'N/A'),
            "Preço Atual": info.get("currentPrice", 0),
            "P/L": info.get("trailingPE", "N/A"),
            "P/VP": info.get("priceToBook", "N/A"),
            "DY (Taxa 12m, %)": dy_pela_taxa,
            "DY 5 Anos Média (%)": info.get("fiveYearAvgDividendYield", "N/A"),
            "Último Dividendo (R$)": info.get("lastDividendValue", "N/A"),
            "Data Últ. Div.": pd.to_datetime(info.get("lastDividendDate"), unit='s') if info.get("lastDividendDate") else "N/A",
            "Data Ex-Div.": pd.to_datetime(info.get("exDividendDate"), unit='s') if info.get("exDividendDate") else "N/A",
            "Payout Ratio (%)": info.get("payoutRatio", 0) * 100,
            "Crescimento Preço (%)": growth_price,
        }

        # LÓGICA CONDICIONAL POR SETOR
        if setor_principal == 'Finance':
            roe = info.get("returnOnEquity", 0) * 100
            if roe > 15:
                roe_score = 25
            elif roe > 12:
                roe_score = 20
            elif roe > 8:
                roe_score = 10
            else:
                roe_score = 0
            dados_especificos = {
                "ROE (%)": roe,
                "Dívida Total": "N/A",
                "Dívida/EBITDA": "N/A",
            }
        else:
            roe = info.get("returnOnEquity", 0) * 100
            if roe > 12:
                roe_score = 15
            elif roe > 8:
                roe_score = 5
            else:
                roe_score = 0
            total_debt = info.get('totalDebt', None)
            ebitda = info.get('ebitda', None)
            divida_ebitda = "N/A" if ebitda is None or ebitda == 0 else total_debt / ebitda
            dados_especificos = {
                "ROE (%)": roe,
                "Dívida Total": total_debt,
                "Dívida/EBITDA": divida_ebitda,
            }

        # Sentimento do Mercado
        try:
            ticker_yf = yf.Ticker(ticker)
            recommendations = ticker_yf.recommendations
            if not recommendations.empty and 'To Grade' in recommendations.columns:
                sentiment = (recommendations['To Grade'].value_counts().reindex(['Strong Buy', 'Buy', 'Hold', 'Sell', 'Strong Sell'], fill_value=0) * [2, 1, 0, -1, -2]).sum() / len(recommendations) if len(recommendations) > 0 else 0
                sentiment_gauge = max(0, min(100, (sentiment * 50 + 50)))  # 0-100
            else:
                sentiment_gauge = 50  # Valor neutro se não houver dados
        except Exception as e:
            print(f"Erro ao buscar sentimento para {ticker}: {e}")
            sentiment_gauge = 50

        dados_detalhados_acoes[ticker] = {**dados_base, **dados_especificos, "Sentimento Gauge": sentiment_gauge}
        print(" ✅")

    except Exception as e:
        print(f" ❌ (Erro: {e})")
        erros.append(ticker)

# --- PASSO 3: ORGANIZAR E SALVAR ---
if dados_detalhados_acoes:
    df_resultado_detalhado = pd.DataFrame.from_dict(dados_detalhados_acoes, orient="index")

    # Formatação
    if "Dívida Total" in df_resultado_detalhado.columns:
        df_resultado_detalhado["Dívida Total"] = df_resultado_detalhado["Dívida Total"].apply(
            lambda x: f"R$ {x/1_000_000_000:.2f} Bi" if pd.notna(x) and isinstance(x, (int, float)) and x != 0 else "N/A"
        )

    colunas_numericas = ["Preço Atual", "P/L", "P/VP"]
    for col in colunas_numericas:
        df_resultado_detalhado[col] = df_resultado_detalhado[col].apply(
            lambda x: f"{x:.2f}" if isinstance(x, (int, float)) else x
        )

    colunas_percentual = ["DY (Taxa 12m, %)", "DY 5 Anos Média (%)", "ROE (%)", "Payout Ratio (%)", "Crescimento Preço (%)", "Sentimento Gauge"]
    for col in colunas_percentual:
        df_resultado_detalhado[col] = df_resultado_detalhado[col].apply(
            lambda x: f"{x:.2f}%" if isinstance(x, (int, float)) else x
        )

    colunas_moeda = ["Último Dividendo (R$)"]
    for col in colunas_moeda:
        df_resultado_detalhado[col] = df_resultado_detalhado[col].apply(
            lambda x: f"R$ {x:.2f}" if isinstance(x, (int, float)) else x
        )

    colunas_data = ["Data Últ. Div.", "Data Ex-Div."]
    for col in colunas_data:
        df_resultado_detalhado[col] = pd.to_datetime(df_resultado_detalhado[col], errors='coerce').dt.strftime('%d-%m-%Y')

    try:
        output_folder = r"E:\finance-manager\data"
        output_filename = "relatorio_analise_b3.csv"
        full_path = os.path.join(output_folder, output_filename)
        os.makedirs(output_folder, exist_ok=True)
        df_resultado_detalhado.to_csv(full_path)
        print(f"\n✅ Relatório salvo em: {full_path}")
    except Exception as e:
        print(f"❌ Erro ao salvar: {e}")

    if erros:
        print(f"⚠️ Ativos com falha: {len(erros)} (ex.: {erros[:5]})")
else:
    print("❌ Nenhum dado coletado.")

print(f"\nProcesso concluído às {datetime.now().strftime('%H:%M')}")

Lendo a lista de ativos do arquivo 'E:\finance-manager\data\scanner_acoes_e_fundos_filtrado.csv'...
✅ 145 ativos serão analisados.

Coletando dados detalhados...
Processando: VIVT3.SA... ✅
Processando: TIMS3.SA... ✅
Processando: FIQE3.SA... ✅
Processando: BRST3.SA... ✅
Processando: DESK3.SA... ✅
Processando: TELB4.SA... ✅
Processando: TELB3.SA... ✅
Processando: OIBR3.SA... ✅
Processando: OIBR4.SA... ✅
Processando: ITUB3.SA... ✅
Processando: ITUB4.SA... ✅
Processando: BPAC3.SA... ✅
Processando: BPAC5.SA... ✅
Processando: BBDC3.SA... ✅
Processando: BBDC4.SA... ✅
Processando: BPAC11.SA... ✅
Processando: SANB11.SA... ✅
Processando: BBAS3.SA... ✅
Processando: ITSA4.SA... ✅
Processando: ITSA3.SA... ✅
Processando: SANB3.SA... ✅
Processando: SANB4.SA... ✅
Processando: B3SA3.SA... ✅
Processando: BBSE3.SA... ✅
Processando: CXSE3.SA... ✅
Processando: RENT3.SA... ✅
Processando: PSSA3.SA... ✅
Processando: MULT3.SA... ✅
Processando: ALOS3.SA... ✅
Processando: BPAN4.SA... ✅
Processando: CYRE3.SA... ✅

In [ ]:
# import pandas as pd

# df_dados = pd.read_excel(r'E:\Github\finance-manager\datasets\relatorio_completo_b3.xlsx')

# print(df_dados.head(2))